Table of Contents:
1. Import Modules and Packes
2. Set-up EPA AQS API
3. Load Data
4. Merge and Examine -  Pandas Dataframe
5. Transform DataFrame 

In [1]:
#1.Imports and Modules
import requests
#store the data we get as a dataframe
import pandas as pd
#convert the response as a strcuctured json
import json
#mathematical operations on lists
import numpy as np
#parse the datetimes we get from NOAA
from datetime import datetime

In [2]:
#2. Set-up EPA AQS API; county codes, parameters, etc for Data Extraction via API

#EPA AQS API: requires specific 'token ID'
#add the access token you got from NOAA
Token = "####"
email = "####@gmail.com"

#Station ID, County ID, State ID...etc needed to run API for data
#Dataset will be performed on 8-hour ozone emission average (O3)

In [3]:
#EPA FIPS County Codes .csv available "states_and_counties.csv"
county_codes = pd.read_csv('C:/Users/srubin/Desktop/Springboard/CapstoneProject2/_states_and_counties.csv')
county_codes.head()

#Acquire appropriate County Codes for top 10 most populous counties in U.S.
top10counties = ['Los Angeles', 'Cook', 'Harris', 'Maricopa','San Diego','Orange','Miami-Dade', 'Dallas', 'Queens','Riverside']
states = ['California', 'Illinois', 'Texas', 'Arizona', 'Florida', 'New York']
county_codes = county_codes[['State Name','State Code','County Code','County Name']]
county_codes.head()

county_codes.columns
county_codes.columns = ['StateName','StateCode','CountyCode','CountyName']
county_codes.head()

top10counties = ['Los Angeles', 'Cook', 'Harris', 'Maricopa','San Diego','Orange','Miami-Dade', 'Dallas', 'Queens','Riverside']
#states = ['California', 'Illinois', 'Texas', 'Arizona', 'Florida', 'New York']
resultsCounties = county_codes[county_codes.StateName.isin(states) & county_codes.CountyName.isin(top10counties)]
resultsCounties

#drop a few extraneous rows
results_Counties = resultsCounties.drop([362,367,1865,2705])
results_Counties
#Here are the County Codes for the top most populous counties in the U.S.!

,StateName,StateCode,CountyCode,CountyName
103,Arizona,04,13,Maricopa
204,California,06,37,Los Angeles
215,California,06,59,Orange
218,California,06,65,Riverside
222,California,06,73,San Diego
610,Illinois,17,31,Cook
1870,New York,36,81,Queens
2581,Texas,48,113,Dallas
2625,Texas,48,201,Harris


In [4]:
#3. Load data via EPA-AQS API
#Function to parse data: email, token, param_code, start_date, end_date, state, county

def request_epa_city_data_data(email, token, param_code, start_date, end_date, state, county):
    """
    :param email: user email info
    :param token: the token assigned to the email
    :param param_code: info being requested
    :param start_date: start date you are requesting
    :param end_date: end date you are requesting
    :param state: us state code
    :param county: county state code
    :return: pandas dataframe of extracted data json object.
    """
    request_url = "https://aqs.epa.gov/data/api/sampleData/byCounty?"+\
                  "email=" +  email + \
                  "&key=" + token + "&param=" + param_code +\
                  "&bdate=" + start_date + \
                  "&edate=" + end_date + "&state=" + state + "&county=" + county
    r_json = requests.get(request_url)
    
    county_data = r_json.json()
    county_data = {key:county_data[key] for key in county_data.keys() & {"Data"} }
    df = pd.concat({k: pd.DataFrame(v) for k, v in county_data.items()})
    return df

#call request_epa_city_data_data for each respective state/county for entire 2019 year, set to unique dataframes
Maricopa_df = request_epa_city_data_data('spencer.max.rubin@gmail.com','orangeheron75','44201', '20190101','20191231',"04","013")
Los_Angeles_df = request_epa_city_data_data('spencer.max.rubin@gmail.com','orangeheron75','44201', '20190101','20191231',"06","037")
Cook_df = request_epa_city_data_data('spencer.max.rubin@gmail.com','orangeheron75','44201', '20190101','20191231',"17","031")
San_Diego_df = request_epa_city_data_data('spencer.max.rubin@gmail.com','orangeheron75','44201', '20190101','20191231',"06","073")
Harris_df = request_epa_city_data_data('spencer.max.rubin@gmail.com','orangeheron75','44201', '20190101','20191231',"48","201")
Orange_df = request_epa_city_data_data('spencer.max.rubin@gmail.com','orangeheron75','44201', '20190101','20191231',"06","059")
Miami_Dade_df = request_epa_city_data_data('spencer.max.rubin@gmail.com','orangeheron75','44201', '20190101','20191231',"12","086")
Dallas_df = request_epa_city_data_data('spencer.max.rubin@gmail.com','orangeheron75','44201', '20190101','20191231',"48","113")
Queens_df = request_epa_city_data_data('spencer.max.rubin@gmail.com','orangeheron75','44201', '20190101','20191231',"36","081")
Riverside_df = request_epa_city_data_data('spencer.max.rubin@gmail.com','orangeheron75','44201', '20190101','20191231',"06","065")

In [5]:
#This is a pd Series for daily mean 1-hour ozone measurements in Maricopa County, AZ.
MaricopaDailyMean = pd.Series(Maricopa_df.groupby(['date_local','time_local','county'])['sample_measurement'].mean())
Los_AngelesDailyMean = pd.Series(Los_Angeles_df.groupby(['date_local','time_local','county'])['sample_measurement'].mean())
CookDailyMean = pd.Series(Cook_df.groupby(['date_local','time_local','county'])['sample_measurement'].mean())
San_DiegoDailyMean = pd.Series(San_Diego_df.groupby(['date_local','time_local','county'])['sample_measurement'].mean())
HarrisDailyMean = pd.Series(Harris_df.groupby(['date_local','time_local','county'])['sample_measurement'].mean())
OrangeDailyMean = pd.Series(Orange_df.groupby(['date_local','time_local','county'])['sample_measurement'].mean())
Miami_DadeDailyMean = pd.Series(Miami_Dade_df.groupby(['date_local','time_local','county'])['sample_measurement'].mean())
DallasDailyMean = pd.Series(Dallas_df.groupby(['date_local','time_local','county'])['sample_measurement'].mean())
RiversideDailyMean = pd.Series(Riverside_df.groupby(['date_local','time_local','county'])['sample_measurement'].mean())
QueensDailyMean = pd.Series(Queens_df.groupby(['date_local','time_local','county'])['sample_measurement'].mean())


In [6]:
#4.Merge and examine pandas dataframe
#combine all county-level dataframes via .concat
AllCountiesMean = pd.concat([MaricopaDailyMean,Los_AngelesDailyMean,CookDailyMean,San_DiegoDailyMean, San_DiegoDailyMean,HarrisDailyMean, OrangeDailyMean, Miami_DadeDailyMean, DallasDailyMean, RiversideDailyMean, QueensDailyMean], axis=1)

cols = []
count = 1
for columns in AllCountiesMean.columns:
    if columns == "sample_measurement":
        cols.append(f'sample_measurement_{count}')
        count +=1
        continue
    cols.append(column)
AllCountiesMean.columns = cols

del AllCountiesMean['sample_measurement_5']
AllCountiesMean.head(10)

sample_measurement_1  sample_measurement_2  \
date_local time_local county                                                    
2019-01-01 00:00      Cook                          NaN                   NaN   
                      Dallas                        NaN                   NaN   
                      Harris                        NaN                   NaN   
                      Los Angeles                   NaN              0.032692   
                      Maricopa                 0.014318                   NaN   
                      Miami-Dade                    NaN                   NaN   
                      Orange                        NaN                   NaN   
                      Queens                        NaN                   NaN   
                      Riverside                     NaN                   NaN   
                      San Diego                     NaN                   NaN   

                                   sample_measurement_3  sample_measurement_4  \
date_local time_local county                                                    
2019-01-01 00:00      Cook                        0.012                   NaN   
                      Dallas                        NaN                   NaN   
                      Harris                        NaN                   NaN   
                      Los Angeles                   NaN                   NaN   
                      Maricopa                      NaN                   NaN   
                      Miami-Dade                    NaN                   NaN   
                      Orange                        NaN                   NaN   
                      Queens                        NaN                   NaN   
                      Riverside                     NaN                   NaN   
                      San Diego                     NaN                 0.026   

                                   sample_measurement_6  sample_measurement_7  \
date_local time_local county                                                    
2019-01-01 00:00      Cook                          NaN                   NaN   
                      Dallas                        NaN                   NaN   
                      Harris                   0.002688                   NaN   
                      Los Angeles                   NaN                   NaN   
                      Maricopa                      NaN                   NaN   
                      Miami-Dade                    NaN                   NaN   
                      Orange                        NaN                 0.039   
                      Queens                        NaN                   NaN   
                      Riverside                     NaN                   NaN   
                      San Diego                     NaN                   NaN   

                                   sample_measurement_8  sample_measurement_9  \
date_local time_local county                                                    
2019-01-01 00:00      Cook                          NaN                   NaN   
                      Dallas                        NaN                 0.017   
                      Harris                        NaN                   NaN   
                      Los Angeles                   NaN                   NaN   
                      Maricopa                      NaN                   NaN   
                      Miami-Dade                 0.0305                   NaN   
                      Orange                        NaN                   NaN   
                      Queens                        NaN                   NaN   
                      Riverside                     NaN                   NaN   
                      San Diego                     NaN                   NaN   

                                   sample_measurement_10  \
date_local time_local county                               
2019-01-01 00:00      C

In [7]:
#5.Transform DataFrame
#rename columns to reflect county/ozone 
AllCountiesMean.rename(columns={'sample_measurement_1':"Ozone_Maricopa", "sample_measurement_2":"Ozone_LosAngeles","sample_measurement_3":"Ozone_Cook","sample_measurement_4":"Ozone_SanDiego","sample_measurement_6":"Ozone_Harris","sample_measurement_7":"Ozone_Orange","sample_measurement_8":'Ozone_MiamiDade',"sample_measurement_9":"Ozone_Dallas", "sample_measurement_10":"Ozone_Riverside","sample_measurement_11":"Ozone_Queens"}, inplace=True)
AllCountiesMean.columns

#reset index 
AllCountiesMean1 = AllCountiesMean.reset_index()
AllCountiesMean2 = AllCountiesMean1.fillna("")
AllCountiesMean2

#Adjusting dataframe dtypes: date, time should be datetime, measurements should be floats
AllCountiesMean2.columns
AllCountiesMean3 = AllCountiesMean2.drop(columns = 'county')
AllCountiesMean3[["Ozone_Maricopa","Ozone_LosAngeles","Ozone_Cook","Ozone_SanDiego","Ozone_Harris","Ozone_Orange","Ozone_MiamiDade","Ozone_Dallas","Ozone_Riverside","Ozone_Queens"]] = AllCountiesMean3[["Ozone_Maricopa","Ozone_LosAngeles","Ozone_Cook","Ozone_SanDiego","Ozone_Harris","Ozone_Orange","Ozone_MiamiDade","Ozone_Dallas","Ozone_Riverside","Ozone_Queens"]].apply(pd.to_numeric)
AllCountiesMean3[["date_local","time_local"]] = AllCountiesMean3[["date_local","time_local"]].apply(pd.to_datetime)
AllCountiesMean3.dtypes

#combining rows so each county measurement is in one row by date and time
AllCountiesMean4 = AllCountiesMean3.groupby(['date_local','time_local']).sum()
AllCountiesMean4.head(30)

AllCountiesMeanOzone = AllCountiesMean4.reset_index()
AllCountiesMeanOzone

,date_local,time_local,Ozone_Maricopa,Ozone_LosAngeles,Ozone_Cook,Ozone_SanDiego,Ozone_Harris,Ozone_Orange,Ozone_MiamiDade,Ozone_Dallas,Ozone_Riverside,Ozone_Queens
0,2019-01-01,2020-10-09 00:00:00,0.014318,0.032692,0.012,0.026000,0.002688,0.039000,0.0305,0.017000,0.035667,0.023
1,2019-01-01,2020-10-09 01:00:00,0.015136,0.028923,0.015,0.022571,0.003312,0.039000,0.0305,0.013000,0.033923,0.017
2,2019-01-01,2020-10-09 02:00:00,0.016955,0.026231,0.018,0.012000,0.002563,0.039667,0.0315,0.011000,0.034769,0.006
3,2019-01-01,2020-10-09 03:00:00,0.021864,0.023846,0.019,0.009000,0.001813,0.039000,0.0320,0.012667,0.035231,0.011
4,2019-01-01,2020-10-09 04:00:00,0.023364,0.021308,0.021,0.028833,0.001313,0.036667,0.0320,0.014000,0.035667,0.012
...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2019-12-31,2020-10-09 19:00:00,0.019045,0.018917,0.028,0.022750,0.009000,0.007000,0.0355,0.001500,0.026769,0.002
8756,2019-12-31,2020-10-09 20:00:00,0.018909,0.018333,0.029,0.020750,0.007812,0.006667,0.0295,0.001000,0.025214,0.007
8757,2019-12-31,2020-10-09 21:00:00,0.017818,0.016417,0.029,0.019625,0.009000,0.008333,0.0190,0.001000,0.021692,0.015
8758,2019-12-31,2020-10-09 22:00:00,0.016000,0.015455,0.028,0.020375,0.007687,0.011333,0.0140,0.001500,0.018917,0.020
